# High-Throughput Materials Screening

This notebook demonstrates how to perform high-throughput analysis on multiple materials.
You will learn how to:

1. Define a list of structures for screening
2. Run batch analysis with progress tracking
3. Collect results in a pandas DataFrame
4. Filter and sort materials by properties
5. Export screening results for further analysis

**Use Cases:**
- Materials discovery and screening
- Systematic property studies
- Database generation
- Benchmarking across materials families

**Prerequisites:**
- CrystalMath with all dependencies (`pip install crystalmath[all]`)
- Access to HPC cluster for parallel execution

## 1. Define Structures for Screening

There are multiple ways to define your screening set:
- List of CIF files
- List of Materials Project IDs
- Query from Materials Project database
- List of pymatgen Structure objects

In [ ]:
# Import required modules
from crystalmath.high_level import HighThroughput
from mp_api.client import MPRester

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

print("Imports successful!")

In [ ]:
# Method 1: Define a list of Materials Project IDs
# Example: Common semiconductors for benchmarking

mp_ids = [
    "mp-149",    # Si (silicon)
    "mp-32",     # Ge (germanium)
    "mp-2534",   # GaAs
    "mp-8062",   # GaN
    "mp-10695",  # InP
    "mp-22598",  # CdTe
    "mp-406",    # GaSb
    "mp-830",    # ZnO
    "mp-2133",   # MoS2
    "mp-1634",   # AlN
]

print(f"Defined {len(mp_ids)} materials for screening")

In [ ]:
# Method 2: Query Materials Project for materials matching criteria
with MPRester() as mpr:
    # Find stable direct-gap semiconductors
    query_results = mpr.materials.summary.search(
        band_gap=(0.5, 3.0),           # Band gap range
        is_stable=True,                 # Thermodynamically stable
        is_gap_direct=True,             # Direct band gap
        nsites=(2, 10),                 # Small unit cells (faster)
        fields=["material_id", "formula_pretty", "band_gap", "nsites"]
    )

print(f"Found {len(query_results)} materials matching criteria")

# Preview results
for doc in query_results[:5]:
    print(f"  {doc.material_id}: {doc.formula_pretty}, gap = {doc.band_gap:.2f} eV")

# Extract mp-ids for screening
queried_mp_ids = [doc.material_id for doc in query_results[:20]]  # Limit to 20

In [ ]:
# Method 3: Define from local CIF files
cif_directory = Path("./structures")

# If directory exists, get all CIF files
if cif_directory.exists():
    cif_files = list(cif_directory.glob("*.cif"))
    print(f"Found {len(cif_files)} CIF files")
else:
    cif_files = []
    print("No local CIF directory found")

## 2. Run Batch Analysis

Run the screening workflow on all defined structures. CrystalMath handles:
- Parallel submission to cluster
- Progress tracking
- Error handling and recovery

In [ ]:
# Simple batch screening using the predefined mp_ids list
screening_set = mp_ids[:5]  # Start with 5 for demonstration

all_results = []
failed_materials = []

print(f"Starting screening of {len(screening_set)} materials...")
print("="*60)

for i, mp_id in enumerate(screening_set):
    print(f"\n[{i+1}/{len(screening_set)}] Processing {mp_id}...")
    
    try:
        # Run analysis for each material
        results = HighThroughput.from_mp(
            material_id=mp_id,
            properties=["relax", "bands"],
            protocol="fast",             # Use fast protocol for screening
            cluster="beefcake2"
        )
        
        # Extract key properties
        all_results.append({
            "mp_id": mp_id,
            "formula": results.formula,
            "space_group": results.space_group,
            "band_gap_ev": results.band_gap_ev,
            "is_direct_gap": results.is_direct_gap,
            "is_metal": results.is_metal,
            "cpu_hours": results.total_cpu_hours
        })
        
        print(f"  -> {results.formula}: gap = {results.band_gap_ev:.3f} eV")
        
    except Exception as e:
        print(f"  -> FAILED: {str(e)[:50]}...")
        failed_materials.append({"mp_id": mp_id, "error": str(e)})

print("\n" + "="*60)
print(f"Screening complete: {len(all_results)} successful, {len(failed_materials)} failed")

In [ ]:
# Advanced: Use async iterator for real-time progress
# This shows progress updates in Jupyter notebooks

import asyncio
from crystalmath.high_level.runners import StandardAnalysis
from crystalmath.high_level.clusters import get_cluster_profile

async def run_screening_async(mp_ids):
    """Run screening with async progress updates."""
    cluster = get_cluster_profile("beefcake2")
    results = []
    
    for mp_id in mp_ids:
        runner = StandardAnalysis(
            cluster=cluster,
            protocol="fast",
            include_relax=False,
            include_bands=True,
            include_dos=False
        )
        
        # Run with progress updates
        async for update in runner.run_async(mp_id):
            print(f"  {update.step_name}: {update.percent:.0f}%", end="\r")
            
            if update.status == "completed" and update.step_name == "complete":
                results.append(update.intermediate_result)
                print(f"\n  -> Completed: {update.intermediate_result.formula}")
    
    return results

# Uncomment to run async version:
# async_results = await run_screening_async(screening_set)

## 3. Collect Results in DataFrame

Organize all results in a pandas DataFrame for analysis.

In [ ]:
# Create DataFrame from results
df = pd.DataFrame(all_results)

print(f"Screening results: {len(df)} materials")
print("\nColumns:", df.columns.tolist())

df

In [ ]:
# Summary statistics
print("Summary Statistics")
print("="*40)
print(f"\nBand Gap (eV):")
print(df['band_gap_ev'].describe())

print(f"\nGap Types:")
print(f"  Direct gaps: {df['is_direct_gap'].sum()} materials")
print(f"  Indirect gaps: {(~df['is_direct_gap']).sum()} materials")
print(f"  Metals: {df['is_metal'].sum()} materials")

In [ ]:
# Enrich with Materials Project data for comparison
with MPRester() as mpr:
    for idx, row in df.iterrows():
        try:
            doc = mpr.materials.summary.get_document_by_id(row['mp_id'])
            df.loc[idx, 'mp_gap'] = doc.band_gap
            df.loc[idx, 'mp_is_direct'] = doc.is_gap_direct
            df.loc[idx, 'e_above_hull'] = doc.energy_above_hull
        except:
            pass

# Calculate error vs MP
df['gap_error'] = df['band_gap_ev'] - df['mp_gap']

print("Enriched DataFrame with MP comparison:")
df[['formula', 'band_gap_ev', 'mp_gap', 'gap_error']]

## 4. Filter and Sort Materials

Apply filters to identify materials meeting specific criteria.

In [ ]:
# Filter: Direct-gap semiconductors with band gap 1-2 eV
solar_candidates = df[
    (df['is_direct_gap'] == True) & 
    (df['band_gap_ev'] >= 1.0) & 
    (df['band_gap_ev'] <= 2.0) &
    (df['is_metal'] == False)
].copy()

print(f"Found {len(solar_candidates)} solar cell candidates (direct gap 1-2 eV):")
solar_candidates[['formula', 'band_gap_ev', 'is_direct_gap']].sort_values('band_gap_ev')

In [ ]:
# Sort by band gap
print("Materials sorted by band gap (ascending):")
df.sort_values('band_gap_ev')[['formula', 'band_gap_ev', 'is_direct_gap', 'space_group']].head(10)

In [ ]:
# Filter: Wide-gap semiconductors (UV applications)
wide_gap = df[
    (df['band_gap_ev'] >= 3.0) &
    (df['is_metal'] == False)
].copy()

print(f"Found {len(wide_gap)} wide-gap semiconductors (> 3 eV):")
wide_gap[['formula', 'band_gap_ev', 'is_direct_gap']]

In [ ]:
# Visualize band gap distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram of band gaps
ax1 = axes[0]
semiconductors = df[~df['is_metal']]
ax1.hist(semiconductors['band_gap_ev'], bins=15, color='steelblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Band Gap (eV)')
ax1.set_ylabel('Count')
ax1.set_title('Band Gap Distribution')
ax1.axvline(x=1.12, color='red', linestyle='--', label='Si (1.12 eV)')
ax1.legend()

# Direct vs indirect gap comparison
ax2 = axes[1]
direct = semiconductors[semiconductors['is_direct_gap'] == True]['band_gap_ev']
indirect = semiconductors[semiconductors['is_direct_gap'] == False]['band_gap_ev']

bp = ax2.boxplot([direct.dropna(), indirect.dropna()], labels=['Direct Gap', 'Indirect Gap'])
ax2.set_ylabel('Band Gap (eV)')
ax2.set_title('Direct vs Indirect Gap Materials')

plt.tight_layout()
plt.savefig('screening_distribution.png', dpi=300)
plt.show()

In [ ]:
# Parity plot: Calculated vs MP database values
if 'mp_gap' in df.columns:
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Scatter plot
    valid = df.dropna(subset=['band_gap_ev', 'mp_gap'])
    ax.scatter(valid['mp_gap'], valid['band_gap_ev'], s=100, alpha=0.7, edgecolors='black')
    
    # Add labels
    for idx, row in valid.iterrows():
        ax.annotate(row['formula'], (row['mp_gap'], row['band_gap_ev']),
                   xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    # Parity line
    max_val = max(valid['mp_gap'].max(), valid['band_gap_ev'].max())
    ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Parity')
    
    # Error bands
    ax.fill_between([0, max_val], [0, max_val*0.9], [0, max_val*1.1], 
                    alpha=0.1, color='green', label='+/- 10%')
    
    ax.set_xlabel('Materials Project Band Gap (eV)', fontsize=12)
    ax.set_ylabel('CrystalMath Calculated Gap (eV)', fontsize=12)
    ax.set_title('Band Gap Comparison: MP vs Calculated', fontsize=14)
    ax.legend()
    ax.set_aspect('equal')
    
    # Calculate MAE
    mae = valid['gap_error'].abs().mean()
    ax.text(0.05, 0.95, f'MAE = {mae:.3f} eV', transform=ax.transAxes,
           fontsize=12, verticalalignment='top')
    
    plt.tight_layout()
    plt.savefig('screening_parity.png', dpi=300)
    plt.show()

## 5. Export Screening Results

Save results in various formats for further analysis and reporting.

In [ ]:
# Export full results to CSV
output_file = "screening_results.csv"
df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

# Export filtered results
if len(solar_candidates) > 0:
    solar_candidates.to_csv("solar_candidates.csv", index=False)
    print("Solar candidates saved to solar_candidates.csv")

In [ ]:
# Export to Excel with multiple sheets
with pd.ExcelWriter('screening_results.xlsx') as writer:
    df.to_excel(writer, sheet_name='All Results', index=False)
    
    # Add filtered subsets as separate sheets
    semiconductors = df[~df['is_metal']]
    semiconductors.to_excel(writer, sheet_name='Semiconductors', index=False)
    
    if len(solar_candidates) > 0:
        solar_candidates.to_excel(writer, sheet_name='Solar Candidates', index=False)
    
    # Add summary statistics
    summary = df.describe()
    summary.to_excel(writer, sheet_name='Statistics')

print("Results saved to screening_results.xlsx")

In [ ]:
# Export to JSON (for programmatic access)
df.to_json('screening_results.json', orient='records', indent=2)
print("Results saved to screening_results.json")

# Export failed materials for debugging
if failed_materials:
    pd.DataFrame(failed_materials).to_json('failed_materials.json', orient='records', indent=2)
    print(f"Failed materials ({len(failed_materials)}) saved to failed_materials.json")

In [ ]:
# Generate LaTeX table for publication
# Select best candidates for the table
top_candidates = df.nsmallest(10, 'gap_error')[['formula', 'space_group', 'band_gap_ev', 'mp_gap', 'is_direct_gap']]

latex_table = top_candidates.to_latex(
    index=False,
    caption='Band gap screening results for selected semiconductors.',
    label='tab:screening',
    float_format='%.3f',
    column_format='lllccc',
    escape=False
)

with open('screening_table.tex', 'w') as f:
    f.write(latex_table)

print("LaTeX table saved to screening_table.tex")
print("\nPreview:")
print(latex_table[:500] + "...")

In [ ]:
# Generate summary report
report = f"""
# High-Throughput Screening Report

## Overview
- Total materials screened: {len(df)}
- Successful calculations: {len(df)}
- Failed calculations: {len(failed_materials)}

## Results Summary
- Semiconductors: {len(df[~df['is_metal']])} materials
- Metals: {df['is_metal'].sum()} materials
- Direct gap: {df['is_direct_gap'].sum()} materials
- Indirect gap: {(~df['is_direct_gap']).sum()} materials

## Band Gap Statistics
- Mean: {df['band_gap_ev'].mean():.3f} eV
- Median: {df['band_gap_ev'].median():.3f} eV
- Min: {df['band_gap_ev'].min():.3f} eV
- Max: {df['band_gap_ev'].max():.3f} eV

## Solar Cell Candidates (1-2 eV, direct gap)
{len(solar_candidates)} materials identified

## Computational Cost
- Total CPU hours: {df['cpu_hours'].sum():.1f} hours
- Average per material: {df['cpu_hours'].mean():.1f} hours
"""

with open('screening_report.md', 'w') as f:
    f.write(report)

print(report)

## Summary

In this notebook, you learned how to:

1. **Define** screening sets from MP IDs, queries, or local files
2. **Run** batch analysis with error handling
3. **Collect** results in pandas DataFrames
4. **Filter** materials by property criteria
5. **Export** to CSV, Excel, JSON, and LaTeX

**Key Tips for High-Throughput Screening:**
- Use `protocol="fast"` for initial screening
- Follow up with `protocol="moderate"` on promising candidates
- Handle failures gracefully (some structures may not converge)
- Validate results against database values (MP, ICSD, etc.)

**Scaling Considerations:**
- For 100+ materials, use async execution with progress tracking
- Consider checkpointing for very long runs
- Use cluster job arrays for massive parallelism

**Files Created:**
- `screening_results.csv` - Full results table
- `screening_results.xlsx` - Excel workbook with multiple sheets
- `screening_results.json` - JSON for programmatic access
- `screening_table.tex` - LaTeX table for publications
- `screening_report.md` - Summary report
- `screening_distribution.png` - Band gap histogram
- `screening_parity.png` - Parity plot vs MP